# Fundamental tensor decompositions.
### Last modification (13.05.2018)


In this tutorial we provide a theoretical backgound on the fundamental tensor
decompositions of multidimensional arrays and show how these data algorithms
can be used with [hottbox](https://github.com/hottbox/hottbox) through **CPD**, **HOSVD**, **HOOI** and **TTSVD** classes.

More details on **CPD**, **HOSVD**, **HOOI** and **TTSVD** classes can be found on the [documentation page](https://hottbox.github.io/stable/api/hottbox.algorithms.decomposition).

> **Note:** this tutorial assumes that you are familiar with the basics of 
tensor algebra, tensor representaitons in different forms and the corresponding conventional 
notation. If you are new to these topics, check out our previous tutorials:
[tutorial_1](https://github.com/hottbox/hottbox-tutorials/blob/master/1_N-dimensional_arrays_and_Tensor_class.ipynb) and 
[tutorial_2](https://github.com/hottbox/hottbox-tutorials/blob/master/2_Efficient_representations_of_tensors.ipynb).

**Requirements:** ``hottbox>=0.1.2``

**Author:** Ilya Kisil - ilyakisil@gmail.com

In [1]:
import numpy as np
from hottbox.core import Tensor, residual_tensor
from hottbox.algorithms.decomposition import TTSVD, HOSVD, HOOI, CPD
from hottbox.metrics import residual_rel_error

# Fundamental tensor decompositions and their implementation

The following algorithms have been implemented in **``hottbox>=0.1.2``**:

- CPD: produces instance of **TensorCPD** class
- HOSVD: produces instance of **TensorTKD** class
- HOOI: produces instance of **TensorTKD** class
- TTSVD: produces instance of **TensorTT** class

> **Note:** more background is coming soon

In this tutorial we use the following randomly generated $3$-rd order tensor for our examples.

In [2]:
np.random.seed(0)
I, J, K = 5, 6, 7

# array_3d = np.arange(I * J * K).reshape((I, J, K)).astype(np.float)
array_3d = np.random.rand(I * J * K).reshape((I, J, K)).astype(np.float)

tensor = Tensor(array_3d)

# Canonical Polyadic Decomposition (CPD)
![tensorcpd](./images/TensorCPD.png)

The CPD decomposition (also referred to as PARAFAC or CANDECOMP) factorizes an $N$-th order tensor $\mathbf{\underline{X}} \in \mathbb{R}^{I_1 \times I_2 \times \cdots \times I_N}$ into a linear combination of terms $\mathbf{b}_r^{(1)} \circ \mathbf{b}_r^{(2)} \circ \cdots \circ \mathbf{b}_r^{(N)}$, which are rank-$1$ tensors. In other words the tensor $\mathbf{\underline{X}}$ is decomposed as

\begin{equation}
\begin{aligned}
\mathbf{\underline{X}} & \simeq \sum_{r=1}^{R} \lambda_r \mathbf{a}_r^{(1)} \circ \mathbf{a}_r^{(2)} \circ \cdots \circ \mathbf{a}_r^{(N)}\\
& = \mathbf{\underline{\Lambda}} \times_1 \mathbf{\underline{A}}^{(1)} \times_2 \mathbf{\underline{A}}^{(2)} \cdots \times_N \mathbf{\underline{A}}^{(N)} \\
& = \Big[    \mathbf{\underline{\Lambda}} ;  \mathbf{\underline{A}}^{(1)} ,  \mathbf{\underline{A}}^{(2)}, \dots, \mathbf{\underline{A}}^{(N)}         \Big]
\end{aligned}
\end{equation}
where $\mathbf{\underline{A}^{(N)}}  = \Big[    \mathbf{a}^{(n)}_1 \hspace{2mm} \mathbf{a}^{(n)}_2  \cdots \mathbf{a}^{(n)}_R   \Big] $ i.e. the concatenation of the corresponding vectors. In case of $3$-rd order tensors, for convention, we let $ \mathbf{A} \colon = \mathbf{A}^{(1)}$,  $\mathbf{A} \colon = \mathbf{A}^{(2)}$, $\mathbf{C} \colon = \mathbf{A}^{(3)}$. $\mathbf{\underline{\Lambda}}$ is an $N$-th order tensor having $\lambda_r$ as entries in positions $\mathbf{\underline{\Lambda}}(i_1, i_2, \dots, i_N)$, where $i_1 = i_2 = \cdots = i_N$, and zeroes elsewhere. 


In **``hottbox``** this form is available through the **``TensorCPD``** class.
In order to create such object, you have 2 options:


1) (See $\textit{Efficient representation of multidimensional arrays}$
) Pass a list of factor matrices (2d ``numpy`` arrays) and a vector of values (as 1d ``numpy`` array) for the main diagonal:

```python
tensor_cpd = TensorCPD(fmat=[A, B, C], core_values=values)
```

2) Decompose an original tensor using class CPD

In this tutorial, we focus on point (2).


To decompose a tensor using the CPD decomposition we create an instance of the CPD class and set a Kruskal rank $R$. The Kruskal rank is passed as a tuple so to keep the same format with other tensor decompositions.

In [3]:
cpd = CPD()
R = (5,)

tensor_cpd = cpd.decompose(tensor, R)
type(tensor_cpd)

hottbox.core.structures.TensorCPD

A **``TensorCPD``** object contains the $\mathbf{\underline{\Lambda}}$ values stored in property **``core``**, while the factor matrices $\mathbf{A}^{(n)}$ are stored in property **``fmat``**.

In [13]:
print('Factor matrices')
for mode, fmat in enumerate(tensor_cpd.fmat):
    print('Mode-{} factor matrix is of shape {}'.format(mode, fmat.shape))
    
print('\nCore tensor')
print(type(tensor_cpd.core))
tensor_cpd.core.describe()
tensor_cpd.core.data

Factor matrices
Mode-0 factor matrix is of shape (5, 5)
Mode-1 factor matrix is of shape (6, 5)
Mode-2 factor matrix is of shape (7, 5)

Core tensor
<class 'hottbox.core.structures.Tensor'>
This tensor is of order 3, consists of 125 elements and its Frobenious norm = 2.24.
Sizes and names of its modes are (5, 5, 5) and {0: 'mode-0', 1: 'mode-1', 2: 'mode-2'} respectively.


array([[[1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1.]]])

In order to convert **``TensorCPD``** into the full representation, simply call: 

```python
tensor_cpd.reconstruct
```

This returns an object of the **``Tensor``** class with N-dimensional array calculated as 
described above and being assinged to the **``_data``** attibute.

In [15]:
tensor_full = tensor_cpd.reconstruct

print(type(tensor_full))
tensor_full.describe()
tensor_full.data

<class 'hottbox.core.structures.Tensor'>
This tensor is of order 3, consists of 210 elements and its Frobenious norm = 7.89.
Sizes and names of its modes are (5, 6, 7) and {0: 'mode-0', 1: 'mode-1', 2: 'mode-2'} respectively.


array([[[ 0.4601365 ,  0.38423614,  0.48859033,  0.62131001,
          0.54691852,  0.58361213,  0.49159456],
        [ 0.67292687,  0.59403672,  0.72678594,  0.81013743,
          0.55605843,  0.32617485,  0.70367859],
        [ 0.18032852,  0.2861196 ,  0.27910407,  0.76039979,
          0.48426828,  0.95862907,  0.76088009],
        [ 0.94275828,  0.47486318,  0.84605741,  0.22145674,
          0.74691123,  0.33027761,  0.85913182],
        [ 0.45441219,  0.43541838,  0.44517452,  0.96472375,
          0.58478457,  0.73463438,  0.17714534],
        [ 0.45427474,  0.46247196,  0.48368908,  0.86823801,
          0.42910584,  0.48006745,  0.61487742]],

       [[ 0.47081601,  0.39236953,  0.48652466,  0.50159374,
          0.34103943,  0.12672599,  0.42371199],
        [ 0.29345678,  0.40894483,  0.37908318,  0.78503303,
          0.27212222,  0.27169589,  0.40563341],
        [ 0.4420933 ,  0.29975187,  0.45217461,  0.22632031,
          0.27859889,  0.13801009,  0.94171161],
        

The **``TensorCPD``** object also contains general information about the underlying tensor, such as its **``rank``** and **``order``**.

> **Note:** The **``rank``** is returned as a tuple. Select its first element to have it as an integer.


In [23]:
R = tensor_cpd.rank
N = tensor_cpd.order

print('The rank of the underlying tensor is {}, and the order is {}'.format(R[0],N))

The rank of the underlying tensor is 5, and the order is 3


# Tucker Decomposition

![tensortkd](./images/TensorTKD.png)

For a tensor $\mathbf{\underline{X}} \in \mathbb{R}^{I \times J \times K}$ illustrated above, the tucker form
represents it as a dense core tensor $\mathbf{\underline{G}}$ with multi-linear rank ($Q, R, P$) and a set of
factor matrices $\mathbf{A} \in \mathbb{R}^{I \times Q}, \mathbf{B} \in \mathbb{R}^{J \times R}$ and $\mathbf{C} \in
\mathbb{R}^{K \times P}$.

The tucker form of a tensor is closely related to the CP form and can be expressed through a 
sequence of mode-$n$ products in a similar way.

$$
\mathbf{\underline{X}} = \mathbf{\underline{G}} \times_1 \mathbf{A} \times_2 \mathbf{B} \times_3 \mathbf{C} = \Big[\mathbf{\underline{G}}; \mathbf{A}, \mathbf{B}, \mathbf{C} \Big]
$$

In **``hottbox``** this form is available through the **``TensorTKD``** class (see **Efficient representation of multidimensional arrays** for further details on this approach) or can be obtained by deocomposing an original tensor. In this tutorial we focus on the latter.



There exist several methods to decompose a tensor in the Tucker format. The two most used ones are Higher Order Singular Value Decomposition (HOSVD), and Higher Order Orthogonal Iteration (HOOI), represented through the **``HOSVD``** and **``HOOI``** classes respectively.

## Tucker representation through Higher Order Singular Value Decomposition (HOSVD)

Consider an $N$-th order tensor $\mathbf{\underline{X}} \in \mathbb{R}^{I_1 \times I_2 \times \cdots \times I_N}$, decomposed in the Tucker format as

$$
\mathbf{\underline{X}} = \mathbf{\underline{G}} \times_1 \mathbf{A}^{(1)} \times_2 \mathbf{A}^{(2)}  \times_3 \cdots \times_N \mathbf{A}^{(N)}  
$$

The HOSVD is a special case of the Tucker decomposition, in which all the factor matrices $\mathbf{A}^{(n)} \in \mathbb{R}^{I_n \times R_n}$ are contrained to be orthogonal, and are computed by the truncated SVD of the mode-$n$ unfolding of tensor $\mathbf{\underline{X}}$:

$$
\begin{aligned}
\mathbf{X}_{(n)} &= \mathbf{U}^{(n)}  \mathbf{\Sigma}^{(n)} \mathbf{V}^{(n)T} \\
\mathbf{A}^{(n)} &= \mathbf{U}^{(n)}_{1:r}
\end{aligned}
$$

where the subscript $1:r$ indicates the truncated $\mathbf{U}^{(n)}$



In order to decompose a tensor using HOSVD, an instance of the **``HOSVD``** class has to be created. Then, the tensor which has to be deocmposed is passed to the **``HOSVD``** instance along with its multilinear rank. The decomposition returns a **``TensorTKD``** object.

In [26]:
hosvd = HOSVD()
ml_rank = (4,5,6)

tensor_tkd_hosvd = hosvd.decompose(tensor, ml_rank)
type(tensor_tkd_hosvd)

hottbox.core.structures.TensorTKD

A **``TensorTKD``** object contains the $\mathbf{\underline{G}}$ values stored in property **``core``**, while the factor matrices $\mathbf{A}^{(n)}$ are stored in property **``fmat``**.

In [27]:
print('Factor matrices')
for mode, fmat in enumerate(tensor_tkd_hosvd.fmat):
    print('Mode-{} factor matrix is of shape {}'.format(mode, fmat.shape))
    
print('\nCore tensor')
print(type(tensor_tkd_hosvd.core))
tensor_tkd_hosvd.core.describe()
tensor_tkd_hosvd.core.data

Factor matrices
Mode-0 factor matrix is of shape (5, 4)
Mode-1 factor matrix is of shape (6, 5)
Mode-2 factor matrix is of shape (7, 6)

Core tensor
<class 'hottbox.core.structures.Tensor'>
This tensor is of order 3, consists of 120 elements and its Frobenious norm = 8.10.
Sizes and names of its modes are (4, 5, 6) and {0: 'mode-0', 1: 'mode-1', 2: 'mode-2'} respectively.


array([[[ 7.31172042e+00, -1.46645203e-01,  3.53335975e-02,
         -7.88128673e-05,  1.14780463e-02,  4.35025917e-02],
        [-1.04406107e-01, -5.14904045e-01, -1.92002044e-01,
         -3.63977877e-01,  5.16454742e-01,  3.85757488e-02],
        [-3.46163346e-02, -1.49344607e-01, -5.10133374e-01,
          1.70603967e-01, -3.13219291e-02, -2.63294842e-02],
        [ 5.48666961e-02, -4.05708761e-01,  1.13646013e-01,
          2.47707520e-01, -1.60923987e-02, -2.34339916e-01],
        [ 2.31937771e-02,  5.18750873e-02, -7.83860515e-01,
          4.80924253e-01, -3.56305213e-02, -9.29619821e-02]],

       [[ 1.06615255e-01, -3.64312315e-02, -1.11172365e-01,
          1.77369720e-01,  6.87554671e-01, -3.62268250e-02],
        [ 2.34238852e-01,  1.29026380e+00, -1.69728926e-01,
          1.60310421e-01,  2.75719853e-01,  1.80648769e-02],
        [ 2.79885627e-01,  5.67385226e-01,  5.37410036e-01,
         -1.37127413e-01, -3.18144928e-02, -3.20520574e-01],
        [-2.96865619e-01, -3.3

In order to convert **``TensorTKD``** into the full representation, simply call: 

```python
tensor_tkd.reconstruct
```

This returns an object of the **``Tensor``** class with N-dimensional array calculated as 
described above and being assinged to the **``_data``** attibute.

In [31]:
tensor_full = tensor_tkd_hosvd.reconstruct

print(type(tensor_full))
tensor_full.describe()
tensor_full.data

<class 'hottbox.core.structures.Tensor'>
This tensor is of order 3, consists of 210 elements and its Frobenious norm = 8.10.
Sizes and names of its modes are (5, 6, 7) and {0: 'mode-0', 1: 'mode-1', 2: 'mode-2'} respectively.


array([[[ 0.49379477,  0.49100835,  0.38365552,  0.68367283,
          0.54888764,  0.42611998,  0.5427561 ],
        [ 0.75505547,  0.77791596,  0.58515515,  0.92860818,
          0.76335682,  0.39896942,  0.92443275],
        [ 0.14631415,  0.34672075,  0.1109151 ,  0.85108005,
          0.64906075,  0.91733203,  0.94285073],
        [ 0.82828896,  0.47559917,  0.83629282,  0.11499961,
          0.69416814,  0.14801001,  0.87075668],
        [ 0.38394455,  0.4517702 ,  0.35980714,  0.78997045,
          0.41944724,  0.45716109, -0.01746523],
        [ 0.55313908,  0.67628517,  0.5447842 ,  0.90858675,
          0.61376985,  0.39624606,  0.49537961]],

       [[ 0.69761707,  0.37777294,  0.71065422,  0.73184619,
          0.19083283,  0.20350362,  0.17147377],
        [ 0.35406242,  0.41800006,  0.41714766,  0.94156982,
          0.09149978,  0.17724141,  0.25126181],
        [ 0.65637626,  0.18630376,  0.46547082,  0.25855016,
          0.18631341,  0.07767786,  0.67210369],
        

The **``TensorTKD``** object also contains general information about the underlying tensor, such as its **``rank``** and **``order``**.

> **Note:** In the case of **``TensorTKD``**, the **``rank``** refers to the multilinear rank.


In [30]:
mlrank = tensor_tkd_hosvd.rank
N = tensor_tkd_hosvd.order

print('The multilinear rank of the underlying tensor is {}'.format(mlrank))
print('The order of the underlying tensor is {}'.format(N))

The multilinear rank of the underlying tensor is (4, 5, 6)
The order of the underlying tensor is 3


## Tucker representation through Higher Order Orthogonal Iteration (HOOI)

HOOI is another special case of the Tuker format. Like HOSVD, it decomposes a tensor into a core tensor and orthogonal factor matrices. The difference between the two lies in the fact that in HOOI the factor matrices are optimized iteratively using an Alternating Least Squares (ALS) approach. (In practice HOSVD is usually used within HOOI to initialize the factor matrices). In other words, for each mode we compute

$$
\begin{aligned}
&\mathbf{\underline{Y}} = \mathbf{\underline{X}} \times_1 \mathbf{A}^{(1)T} \times_2 \cdots \times_{n-1} \mathbf{A}^{(n-1)T} \times_{n+1} \mathbf{A}^{(n+1)} \times \cdots \times_N \mathbf{A}^{(N)} \\
&\mathbf{A}^{(n)} \leftarrow R_n \text{ leftmost singular vectors of } \mathbf{Y}_{(n)}
\end{aligned}
$$

The above is repeated until convergence, then the core tensor $\mathbf{\underline{G}} \in \mathbb{R}^{R_1 \times R_2 \times \cdots \times R_N}$ is computed as

$$
\mathbf{\underline{G}} = \mathbf{\underline{X}} \times_1 \mathbf{A}^{(1)T}  \times_2 \mathbf{A}^{(2)T} \times_3 \cdots  \times_N \mathbf{A}^{(N)T}
$$


Although the **``hottbox``** implementation HOOI clearly takes the above into account, decomposing a tensor via HOOI follows analogous commands to decomposing it using HOSVD. 

First, an instance of the **``HOOI``** class is created, and an original tensor and the desired multilinear rank are passed to its **``.decompose()``** method. This returns a **``TensorTKD``** object.  




In [32]:
hooi = HOOI()
ml_rank = (4,5,6)

tensor_tkd_hooi = hosvd.decompose(tensor, ml_rank)
type(tensor_tkd_hooi)

hottbox.core.structures.TensorTKD

A **``TensorTKD``** object contains the $\mathbf{\underline{G}}$ values stored in property **``core``**, while the factor matrices $\mathbf{A}^{(n)}$ are stored in property **``fmat``**.

In [33]:
print('Factor matrices')
for mode, fmat in enumerate(tensor_tkd_hooi.fmat):
    print('Mode-{} factor matrix is of shape {}'.format(mode, fmat.shape))
    
print('\nCore tensor')
print(type(tensor_tkd_hooi.core))
tensor_tkd_hooi.core.describe()
tensor_tkd_hooi.core.data

Factor matrices
Mode-0 factor matrix is of shape (5, 4)
Mode-1 factor matrix is of shape (6, 5)
Mode-2 factor matrix is of shape (7, 6)

Core tensor
<class 'hottbox.core.structures.Tensor'>
This tensor is of order 3, consists of 120 elements and its Frobenious norm = 8.10.
Sizes and names of its modes are (4, 5, 6) and {0: 'mode-0', 1: 'mode-1', 2: 'mode-2'} respectively.


array([[[-7.31172042e+00,  1.46645203e-01, -3.53335975e-02,
          7.88128673e-05,  1.14780463e-02, -4.35025917e-02],
        [ 1.04406107e-01,  5.14904045e-01,  1.92002044e-01,
          3.63977877e-01,  5.16454742e-01, -3.85757488e-02],
        [ 3.46163346e-02,  1.49344607e-01,  5.10133374e-01,
         -1.70603967e-01, -3.13219291e-02,  2.63294842e-02],
        [-5.48666961e-02,  4.05708761e-01, -1.13646013e-01,
         -2.47707520e-01, -1.60923987e-02,  2.34339916e-01],
        [ 2.31937771e-02,  5.18750873e-02, -7.83860515e-01,
          4.80924253e-01,  3.56305213e-02, -9.29619821e-02]],

       [[ 1.06615255e-01, -3.64312315e-02, -1.11172365e-01,
          1.77369720e-01, -6.87554671e-01, -3.62268250e-02],
        [ 2.34238852e-01,  1.29026380e+00, -1.69728926e-01,
          1.60310421e-01, -2.75719853e-01,  1.80648769e-02],
        [ 2.79885627e-01,  5.67385226e-01,  5.37410036e-01,
         -1.37127413e-01,  3.18144928e-02, -3.20520574e-01],
        [-2.96865619e-01, -3.3

In order to convert **``TensorTKD``** into the full representation, simply call: 

```python
tensor_tkd.reconstruct
```

This returns an object of the **``Tensor``** class with N-dimensional array calculated as 
described above and being assinged to the **``_data``** attibute.

In [37]:
tensor_full = tensor_tkd_hooi.reconstruct

print(type(tensor_full))
tensor_full.describe()
tensor_full.data

<class 'hottbox.core.structures.Tensor'>
This tensor is of order 3, consists of 210 elements and its Frobenious norm = 8.10.
Sizes and names of its modes are (5, 6, 7) and {0: 'mode-0', 1: 'mode-1', 2: 'mode-2'} respectively.


array([[[ 0.49379477,  0.49100835,  0.38365552,  0.68367283,
          0.54888764,  0.42611998,  0.5427561 ],
        [ 0.75505547,  0.77791596,  0.58515515,  0.92860818,
          0.76335682,  0.39896942,  0.92443275],
        [ 0.14631415,  0.34672075,  0.1109151 ,  0.85108005,
          0.64906075,  0.91733203,  0.94285073],
        [ 0.82828896,  0.47559917,  0.83629282,  0.11499961,
          0.69416814,  0.14801001,  0.87075668],
        [ 0.38394455,  0.4517702 ,  0.35980714,  0.78997045,
          0.41944724,  0.45716109, -0.01746523],
        [ 0.55313908,  0.67628517,  0.5447842 ,  0.90858675,
          0.61376985,  0.39624606,  0.49537961]],

       [[ 0.69761707,  0.37777294,  0.71065422,  0.73184619,
          0.19083283,  0.20350362,  0.17147377],
        [ 0.35406242,  0.41800006,  0.41714766,  0.94156982,
          0.09149978,  0.17724141,  0.25126181],
        [ 0.65637626,  0.18630376,  0.46547082,  0.25855016,
          0.18631341,  0.07767786,  0.67210369],
        

The **``TensorTKD``** object also contains general information about the underlying tensor, such as its **``rank``** and **``order``**.

> **Note:** In the case of **``TensorTKD``**, the **``rank``** refers to the multilinear rank.

In [36]:
mlrank = tensor_tkd_hooi.rank
N = tensor_tkd_hooi.order

print('The multilinear rank of the underlying tensor is {}'.format(mlrank))
print('The order of the underlying tensor is {}'.format(N))

The multilinear rank of the underlying tensor is (4, 5, 6)
The order of the underlying tensor is 3


## Tensor Train Decomposition via SVD

![tensortt](./images/TensorTT.png)

Tensor trains (TTs) are the simplest kinds of tensor networks, i.e. a decomposition of a high-order tensor in a set of sparsely interconnected lower-order tensors and factor matrices. Mathematically, an $N$-th order tensor $\mathbf{\underline{X}} \in \mathbb{R}^{I_1 \times I_2 \times \cdots \times I_N}$ can be expressed as a TT as

\begin{equation}
\mathbf{\underline{X}} = \mathbf{A} \times^1_2 \mathbf{\underline{G}}^{(1)}  \times^1_3 \mathbf{\underline{G}}^{(2)}   \times^1_3 \cdots \times^1_3 \mathbf{\underline{G}}^{(N-1)} \times^1_3 \mathbf{B} = \Big[  \mathbf{A}, \mathbf{\underline{G}}^{(1)}, \mathbf{\underline{G}}^{(2)}, \cdots, \mathbf{\underline{G}}^{(N-1)}, \mathbf{B}  \Big]
\end{equation}

Each element of a TT is generally referred to as $\textit{core}$, and $\mathbf{A} \in \mathbb{R}^{I_1 \times R_1}$, $\mathbf{B} \in \mathbb{R}^{R_{N-1}\times I_N}$, $\mathbf{\underline{G}}^{(n)} \in \mathbb{R}^{R_n \times I_{n+1} \times R_{n+1}}$

In **``hottbox``** this form is available through the **``TensorTT``** class (see **Efficient representation of multidimensional arrays**) or via a decomposition employing the Tensor Train SVD (TTSVD) algorithm. In this tutorial we focus on the latter. 

The TTSVD algorithm involves iteratively performing a series of foldings and unfoldings on an original tensor $\mathbf{\underline{X}} \in \mathbb{R}^{I_1 \times I_2 \times \cdots \times I_N}$ in conjunction with SVD. At every iteration a core $\mathbf{\underline{G}}^{(n)} \in \mathbb{R}^{R_n \times I_{n+1} \times R_{n+1}}$ is computed, where the TT-rank of the TT $(R_1, R_2, \dots, R_N)$ has been specified a priori by the user. 

To decompose a tensor $\mathbf{\underline{X}}$ in a TT via the TTSVD algorithm, similarly with HOSVD and HOOI, we first create an instance of the class **``TTSVD``**. Within this instance the **``.decompose()``** method is called, which receives as parameters the original tensor and the TT-rank. The decomposition returns a **``TensorTT``** object.

In [39]:
tt = TTSVD()
tt_rank = (2,3)

tensor_tt = tt.decompose(tensor, tt_rank)
type(tensor_tt)

hottbox.core.structures.TensorTT

A **``TensorTT``** object contains the cores $\mathbf{\underline{G}}^{(n)}$ stored in the property **``cores``**. Specifically, it returns a list containing all the cores.

In [46]:
print('Core tensors')
for n, core in enumerate(tensor_tt.cores):
    print('Core {} of shape {}'.format(n, core.shape))

Core tensors
Core 0 of shape (5, 2)
Core 1 of shape (2, 6, 3)
Core 2 of shape (3, 7)


A specific core $n$ can also be accessed via the method **``.core()``** which receives $n$ as parameter


```python
tensor_tt.core(n)
```


In [45]:
desired_core = tensor_tt.core(1)
desired_core.describe()
desired_core.data

This tensor is of order 3, consists of 36 elements and its Frobenious norm = 1.73.
Sizes and names of its modes are (2, 6, 3) and {0: 'mode-0', 1: 'mode-1', 2: 'mode-2'} respectively.


array([[[-3.45363942e-01,  6.91633539e-02,  1.47163743e-01],
        [-3.99625113e-01, -7.72049028e-02, -3.49687378e-01],
        [-4.27100595e-01, -2.84755430e-01,  3.20261689e-01],
        [-4.30849457e-01,  3.10689814e-01,  3.92590104e-01],
        [-4.02938580e-01, -2.73067115e-04, -9.96401253e-02],
        [-4.27938190e-01, -1.03592524e-01, -3.65245864e-01]],

       [[-3.30732887e-02,  2.97014571e-01, -2.02644734e-01],
        [ 2.27815028e-04, -4.88289574e-03, -5.28279451e-01],
        [-2.02657155e-02,  4.35268706e-01, -1.20441021e-01],
        [ 1.97574567e-02, -7.18324111e-01, -2.81796432e-02],
        [-5.68375471e-02,  5.93854499e-02,  7.67481314e-02],
        [ 4.85931217e-02, -6.07785215e-02,  3.38673680e-01]]])

In order to convert **``TensorTT``** into the full representation, simply call: 

```python
tensor_tt.reconstruct
```

This returns an object of the **``Tensor``** class with N-dimensional array calculated as 
described above and being assinged to the **``_data``** attibute.

In [47]:
tensor_full = tensor_tt.reconstruct

print(type(tensor_full))
tensor_full.describe()
tensor_full.data

<class 'hottbox.core.structures.Tensor'>
This tensor is of order 3, consists of 210 elements and its Frobenious norm = 7.64.
Sizes and names of its modes are (5, 6, 7) and {0: 'mode-0', 1: 'mode-1', 2: 'mode-2'} respectively.


array([[[0.46083185, 0.31840353, 0.43962288, 0.400696  , 0.57679603,
         0.52624231, 0.46231584],
        [0.55629843, 0.45720105, 0.60233867, 0.64303109, 0.5900637 ,
         0.50276186, 0.52787145],
        [0.47415982, 0.35519694, 0.43450463, 0.6531003 , 0.76637386,
         0.89193904, 0.4994085 ],
        [0.85793225, 0.4996603 , 0.82086532, 0.16499666, 0.76405806,
         0.3114782 , 0.82580747],
        [0.52428703, 0.44339765, 0.58317602, 0.62055065, 0.52271656,
         0.42259184, 0.4898069 ],
        [0.61441113, 0.60325826, 0.75805194, 0.94085401, 0.5268105 ,
         0.41117134, 0.54527913]],

       [[0.53113227, 0.39632565, 0.59227679, 0.29836595, 0.35314379,
         0.04773239, 0.47630675],
        [0.33248824, 0.47063949, 0.5423968 , 0.86325978, 0.12052702,
         0.04616461, 0.2428014 ],
        [0.58322677, 0.39444659, 0.60298902, 0.26777147, 0.4817917 ,
         0.1915537 , 0.54563666],
        [0.17026671, 0.18441375, 0.1467585 , 0.62061127, 0.5013082 ,
  

The **``TensorTT``** object also contains general information about the underlying tensor, such as its **``rank``** and **``order``**.

> **Note:** In the case of **``TensorTT``**, the **``rank``** refers to the  TT-rank.

In [48]:
mlrank = tensor_tt.rank
N = tensor_tt.order

print('The TT-rank of the underlying tensor is {}'.format(mlrank))
print('The order of the underlying tensor is {}'.format(N))

The TT-rank of the underlying tensor is (2, 3)
The order of the underlying tensor is 3


## Evaluating results of tensor decompositions

For each result of the tensor decomposition we can compute a residual tensor and calculate relative error of approximation:
```python
    tensor_res = residual_tensor(tensor, tensor_cpd)
    rel_error = tensor_res.frob_norm / tensor.frob_norm        
```
Or can do it in one line:
```python
    rel_error = residual_rel_error(tensor, tensor_cpd)
```



In [7]:
tensor_cpd_res = residual_tensor(tensor, tensor_cpd)
print('Residual tensor is instance of {}'.format(type(tensor_cpd_res)))

Residual tensor is instance of <class 'hottbox.core.structures.Tensor'>


In [8]:
rel_error = tensor_cpd_res.frob_norm / tensor.frob_norm 
print('Relative error of CPD approximation = {:.2f}'.format(rel_error))

rel_error = residual_rel_error(tensor, tensor_cpd)
print('Relative error of CPD approximation = {:.2f}'.format(rel_error))

Relative error of CPD approximation = 0.31
Relative error of CPD approximation = 0.31


In [9]:
rel_error = residual_rel_error(tensor, tensor_tkd_hosvd)
print('Relative error of HOSVD approximation = {:.2f}'.format(rel_error))

Relative error of HOSVD approximation = 0.21


In [10]:
rel_error = residual_rel_error(tensor, tensor_tkd_hooi)
print('Relative error of HOOI approximation = {:.2f}'.format(rel_error))

Relative error of HOOI approximation = 0.21


In [11]:
rel_error = residual_rel_error(tensor, tensor_tt)
print('Relative error of TT approximation = {:.2f}'.format(rel_error))

Relative error of TT approximation = 0.39
